In [1]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = user_secrets.get_secret("HUGGINGFACEHUB_API_TOKEN")
os.environ['WANDB_API_KEY'] = user_secrets.get_secret("WANDB_API_KEY ")

from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACEHUB_API_TOKEN")
login(token = hf_token)

In [2]:
!git clone https://github.com/namwasinyourheart/ft-llm/

Cloning into 'ft-llm'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (92/92), done.:  66% (61/92)
remote: Total 131 (delta 71), reused 96 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (131/131), 71.80 KiB | 2.56 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [3]:
cd ft-llm

/kaggle/working/ft-llm


In [4]:
!pip install -r requirements.txt -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 3.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 70.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.9 MB/s eta 0:00:00
   ━

# GENERATION

In [10]:
%%writefile generate.py

from src.utils.model_utils import get_model_tokenizer
from typing import List, Tuple

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Pipeline,
    PreTrainedModel,
    PreTrainedTokenizer,
)

from transformers import BitsAndBytesConfig
from src.utils.model_utils import set_torch_dtype_and_attn_implementation, get_quantization_config


def get_quantization_config(model_args                          
) -> BitsAndBytesConfig | None:
    if model_args['load_in_4bit']:
        # compute_dtype = torch.float16
        # if torch_dtype not in {"auto", None}:
        #     compute_dtype = getattr(torch, model_args.torch_dtype)
        torch_dtype, attn_implementation = set_torch_dtype_and_attn_implementation()

        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            # bnb_4bit_compute_dtype=model_args['bnb_4bit_compute_dtype'],
            bnb_4bit_compute_dtype=torch_dtype,
            bnb_4bit_quant_type=model_args['bnb_4bit_quant_type'],
            bnb_4bit_use_double_quant=model_args['bnb_4bit_use_double_quant'],
            bnb_4bit_quant_storage=model_args['bnb_4bit_quant_storage'],
        ).to_dict()
    elif model_args['load_in_8bit']:
        quantization_config = BitsAndBytesConfig(
            load_in_8bit=True,
        ).to_dict()
    else:
        quantization_config = None

    return quantization_config


def load_model_for_generate(model_args, device_args):

    # Set torch_dtype and attn_implementation
    torch_dtype, attn_implementation = set_torch_dtype_and_attn_implementation()

    # QLora Config
    quantization_config = get_quantization_config(model_args)

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_args['pretrained_model_name_or_path'],
        trust_remote_code=True,
        torch_dtype=torch_dtype,
        quantization_config=quantization_config if not device_args['use_cpu'] else None,
        device_map="cpu" if device_args['use_cpu'] else "auto",
        attn_implementation=attn_implementation,
        # low_cpu_mem_usage=True if not use_cpu else False
    )

    return model

def load_model_vllm(model_args, device_args):
    # Set torch_dtype and attn_implementation
    torch_dtype, attn_implementation = set_torch_dtype_and_attn_implementation()

    # Load model
    llm = LLM(
        model=model_args['pretrained_model_name_or_path'],
        dtype=torch_dtype
        )

    return llm


def load_tokenizer_for_generate(
    model_args
) -> PreTrainedTokenizer:
    
    tokenizer = AutoTokenizer.from_pretrained(model_args['pretrained_tokenizer_name_or_path'])

    return tokenizer

def prepare_prompt(prompt_args):
    if prompt_args['use_no_keys']:
        return prompt_args['input']['text']

    intro = f"{prompt_args['intro_text']}"
    instruction = f"{prompt_args['instruction']['key']}\n{prompt_args['instruction']['text']}"

    examples = None
    if prompt_args['use_examples'] and 'examples' in prompt_args:
        example_template = prompt_args['examples']['template']
        formatted_examples = "\n\n".join(
            example_template.format(**example) for example in prompt_args['examples']['list']
        )
        examples = f"{prompt_args['examples']['key']}\n{formatted_examples}"

    input = f"{prompt_args['input']['key']}\n{prompt_args['input']['text']}"

    context = None
    if prompt_args['use_context'] and prompt_args['context']['key']:
        context = f"{prompt_args['context']['key']}\n{prompt_args['context']['text']}"

    response_key = f"{prompt_args['response_key']}\n"

    # Collect all non-empty parts and join them
    parts = [part for part in [intro, instruction, examples, context, input, response_key] if part]

    return "\n\n".join(parts)

def postprocess(prompt: str, tokenizer, output: str, response_key: str, end_key: str, return_full_text: bool=False):
    
    import re
    decoded = None
    fully_decoded = output
    
    if not end_key:
        end_key = tokenizer.eos_token
    
    pattern = r".*?{}\s*(.+?){}".format(response_key, end_key)
    m = re.search(pattern, fully_decoded, flags=re.DOTALL)

    if m:
        decoded = m.group(1).strip()
    else:
        # The model might not generate the end_key sequence before reaching the max tokens. In this case,
        # return everything after response_key.
        pattern = r".*?{}\s*(.+)".format(response_key)
        m = re.search(pattern, fully_decoded, flags=re.DOTALL)
        if m:
            decoded = m.group(1).strip()
        else:
            print(f"Failed to find response in:\n{fully_decoded}")
            
    if return_full_text:
        decoded = f"{prompt}{decoded}"

    return decoded


def generate_response(model_args, prompt_args, gen_args, device_args):
    model = load_model_for_generate(model_args, device_args)
    
    tokenizer = load_tokenizer_for_generate(model_args)

    prompt = prepare_prompt(prompt_args)

    input = tokenizer(prompt, return_tensors='pt', padding=False, truncation=True)
    input_ids = input.input_ids

    input_ids = input_ids.to(model.device)
    input = input.to(model.device)

    output_ids = model.generate(**input, 
                                max_new_tokens=gen_args['max_new_tokens'], 
                                pad_token_id=tokenizer.pad_token_id)
    
    output = tokenizer.decode(output_ids[0], 
                              skip_special_tokens=True)

    if gen_args['do_postprocess']:
        # outputs = [postprocess(prompt, tokenizer, output, return_full_text, **generate_kwargs) for (prompt, output) in zip(prompts, outputs)]

        output = postprocess(prompt, tokenizer, output, 
                             prompt_args['response_key'],
                             prompt_args['end_key'], 
                             gen_args['return_full'])
    
    return output


def parse_args():
    import argparse
    parser = argparse.ArgumentParser(description="Load generation config.")
    parser.add_argument("--config_path", type=str, required=True, help="Path to the YAML config file for generating.")
    parser.add_argument("--input_text", type=str, default=None, help="The text input (question).")

    return parser.parse_args()

if __name__ == "__main__":

    from omegaconf import OmegaConf
    args = parse_args()

    # Load the generation config file
    cfg = OmegaConf.load(args.config_path)

    model_args = cfg.model
    prompt_args = cfg.prompt
    gen_args = cfg.generate
    device_args = cfg.device

    if args.input_text:
        prompt_args['input']['text'] = args.input_text

    response = generate_response(model_args, prompt_args, gen_args, device_args)
    print(response)

Overwriting generate.py


In [55]:
%%writefile configs/generate.yaml
model:
  pretrained_model_name_or_path: meta-llama/Llama-3.2-1B-Instruct # meta-llama/Llama-3.2-1B-Instruct #"Qwen/Qwen2.5-Math-1.5B-Instruct"
  pretrained_tokenizer_name_or_path: meta-llama/Llama-3.2-1B-Instruct # meta-llama/Llama-3.2-1B-Instruct # "Qwen/Qwen2.5-Math-1.5B-Instruct"
  load_in_4bit: true
  load_in_8bit: false
  bnb_4bit_compute_dtype: null
  bnb_4bit_quant_type: "nf4"
  bnb_4bit_use_double_quant: false
  bnb_4bit_quant_storage: "uint8"

prompt:
    use_no_keys: false
    use_examples: false
    use_context: false
    intro_text: 

    instruction:
        key: "### Instruction:"
        text: "Summarize the following dialogue:"
    
    context:
        key: null
        text: null

    input:
        key: "### Dialogue:"
        text: "Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

    
    response_key: "### Summary:"
    end_key: null


generate:
  use_vllm: False
  do_postprocess: false
  return_full: false
  skip_special_tokens: true
  max_new_tokens: 512
  temperature: 0
  do_sample: null
  top_p: null
  # num_beams: 4,    
  # num_return_sequences: 4,
  # return_dict_in_generate: True,
  # output_scores: True,
  

device:
  use_cpu: False


Overwriting configs/generate.yaml


In [58]:
!python generate.py --config_path=configs/generate.yaml

2025-02-23 15:41:50.529217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-23 15:41:50.551320: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-23 15:41:50.557877: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
None

### Instruction:
Summarize the following dialogue:

### Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
H